In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, optimizers, losses, metrics
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import cv2 as cv
import os
import pickle

In [2]:
train_directory = r"C:\Users\Roberto\Documents\Codes\ML_API\Datasets\Train"
test_directory = r"C:\Users\Roberto\Documents\Codes\ML_API\Datasets\Test"
batch_size = 32
img_size = (224, 224)
img_shape = (*img_size, 3)

In [3]:
train_set = keras.utils.image_dataset_from_directory(
                        train_directory,
                        labels='inferred',
                        color_mode='rgb',
                        batch_size=batch_size,
                        image_size=img_size,
                        seed=123,
                        validation_split=0.35,
                        subset="training")

val_set = keras.utils.image_dataset_from_directory(
                        train_directory,
                        labels='inferred',
                        color_mode='rgb',
                        batch_size=batch_size,
                        image_size=img_size,
                        seed=123,
                        validation_split=0.35,
                        subset="validation")

Found 5410 files belonging to 2 classes.
Using 3517 files for training.
Found 5410 files belonging to 2 classes.
Using 1893 files for validation.


In [9]:
test_set = keras.utils.image_dataset_from_directory(
                        test_directory,
                        labels='inferred',
                        label_mode='binary',
                        color_mode='rgb',
                        batch_size=batch_size,
                        image_size=img_size,
                        seed=123,
                        validation_split=0.001,
                        subset="training")

Found 1056 files belonging to 2 classes.
Using 1055 files for training.


In [10]:
# train_set = train_set.map(lambda x, y: (tf.image.resize(x, img_size), y))
# val_set = val_set.map(lambda x, y: (tf.image.resize(x, img_size), y))
# test_set = test_set.map(lambda x, y: (tf.image.resize(x, img_size), y))

#train_set = train_set.cache().batch(batch_size).prefetch(buffer_size=10)
#val_set = val_set.cache().batch(batch_size).prefetch(buffer_size=10)
# test_set = test_set.cache().batch(batch_size).prefetch(buffer_size=10)

In [4]:
base_model = keras.applications.Xception(include_top=False, weights='imagenet', input_shape=img_shape)
base_model.trainable = False

In [5]:
data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

In [6]:
inputs = keras.Input(shape=(img_shape))
x = data_augmentation(inputs)

rescaling = layers.Rescaling(scale=1/127.5, offset=-1)
x = rescaling(x)

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, output)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

In [7]:
model.compile(
    optimizer=optimizers.Adam(), 
    loss=losses.BinaryCrossentropy(), 
    metrics=[metrics.BinaryAccuracy()])

In [8]:
epochs = 20
history = model.fit(train_set, epochs=epochs, validation_data=val_set)

Epoch 1/20
110/110 [==============================] - 137s 1s/step - loss: 0.3069 - binary_accuracy: 0.9062 - val_loss: 0.1455 - val_binary_accuracy: 0.9762
Epoch 2/20
110/110 [==============================] - 136s 1s/step - loss: 0.1337 - binary_accuracy: 0.9701 - val_loss: 0.0785 - val_binary_accuracy: 0.9947
Epoch 3/20
110/110 [==============================] - 136s 1s/step - loss: 0.0912 - binary_accuracy: 0.9878 - val_loss: 0.0626 - val_binary_accuracy: 0.9968
Epoch 4/20
110/110 [==============================] - 135s 1s/step - loss: 0.0722 - binary_accuracy: 0.9869 - val_loss: 0.0416 - val_binary_accuracy: 0.9984
Epoch 5/20
110/110 [==============================] - 135s 1s/step - loss: 0.0584 - binary_accuracy: 0.9912 - val_loss: 0.0341 - val_binary_accuracy: 0.9979
Epoch 6/20
110/110 [==============================] - 135s 1s/step - loss: 0.0511 - binary_accuracy: 0.9898 - val_loss: 0.0274 - val_binary_accuracy: 0.9979
Epoch 7/20
110/110 [==============================] - 135s

In [9]:
def adj_img(image):
    width = int(224)
    dim = (width, width)
    img_resize = cv.resize(image, dim)
    adjusted_image = np.expand_dims(img_resize, axis=0)
    return adjusted_image

In [119]:
img = cv.imread(r"C:\Users\Roberto\Documents\Codes\ML_API\Datasets\Test\1\YY580.png")
img_resize = adj_img(img)
model.predict(img_resize)[0]

1/1 [==============================] - 0s 59ms/step


array([0.97869027], dtype=float32)

In [53]:
path_0 = r"C:\Users\Roberto\Documents\Codes\ML_API\Datasets\Test\0"
path_1 = r"C:\Users\Roberto\Documents\Codes\ML_API\Datasets\Test\1"

def get_files(path):
    return os.listdir(path)

def get_label(path):
    return int(path[-1])

In [75]:
def abs_error_test(path):
    predictions = []
    #get files
    files = get_files(path)
    #iterate over files
    for i in range(len(files)):
        #return image 
        image = cv.imread(path + '\\' + files[i])
        #adjusts the image to pred
        pred_ready = adj_img(image)
        #returns prediction
        prediction = model.predict(pred_ready)[0][0]
        if prediction >= 0.5:
            prediction = 1
        else:
            prediction = 0
        predictions.append(prediction)
        
    sum_list = sum(predictions)
    correct = get_label(path) * len(files)
    diff = sum_list - correct
    
    return diff, sum_list, correct, predictions

In [76]:
diff0, sum0, correct0, pred0 = abs_error_test(path_0)
diff1, sum1, correct1, pred1 = abs_error_test(path_1)

print(diff0)
print(diff1)

1/1 [==============================] - 0s 54ms/step


1/1 [==============================] - 0s 54ms/step
0
1/1 [==============================] - 0s 54ms/step


1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 47ms/step
-19


In [90]:
new_dict = {}
for ind, el in enumerate(pred1):
    if el == 0:
        new_dict[ind]=el

In [91]:
predictions = np.concatenate( (np.array(pred0), np.array(pred1)), axis=0)

truth0 = np.full( (len(pred0),1), 0, dtype=int)
truth1 = np.full( (len(pred1),1), 1, dtype=int)

truth = np.concatenate( (truth0, truth1), axis=0)

In [97]:
assert(truth.shape[0] == predictions.shape[0])

In [98]:
print(confusion_matrix(truth, predictions))
print(accuracy_score(truth, predictions))

[[489   0]
 [ 19 548]]
0.9820075757575758


In [1]:
#model.save('model.h5')
from tensorflow import keras
loaded_model = keras.models.load_model('../app/model/model.h5')

In [2]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tf_lite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Roberto\AppData\Local\Temp\tmp5eg_9u49\assets


INFO:tensorflow:Assets written to: C:\Users\Roberto\AppData\Local\Temp\tmp5eg_9u49\assets


In [3]:
with open('model.tflite', 'wb') as f:
    f.write(tf_lite_model)

In [6]:
!pip install tflite_runtime

ERROR: Could not find a version that satisfies the requirement tflite_runtime (from versions: none)
ERROR: No matching distribution found for tflite_runtime


In [7]:
import tflite_runtime.interpreter as tflite

model = tflite.Interpreter('model.tflite')
model

ModuleNotFoundError: No module named 'tflite_runtime'

In [ ]:
model.allocate_tensors()

input_details = model.get_input_details()
input_details

In [120]:
loaded_model.predict(img_resize)

1/1 [==============================] - 0s 61ms/step


array([[0.97869027]], dtype=float32)

In [17]:
# def pred_class(prediction):
#     for i in range(len(prediction)):
        
#         if prediction[i][0] >= 0.5:
#             prediction[i] = 1
#         else:
#             prediction[i] = 0
#     return prediction
# adj_pred = pred_class(predict)
# adj_pred.shape

# def destruct_tfds(dataset):
#     final_ds = []
#     img_ds = []
#     #take all batches in data from ds
#     for img, label in dataset.take(tf.data.experimental.cardinality(dataset)):
#         #from each batch, unssemble it and add to final_ds
#         for i in range(label.shape[0]):
#             final_ds.append(int(label[i]))
#             img_ds.append(img[i])
#     return final_ds, img_ds

# final_test_set, img_test_set = destruct_tfds(test_set)

# def find_error(adj_pred, truth):
#     error_data = []
#     #just checking hehe
#     assert(len(adj_pred) == len(truth))
#     #iter over pred-truth
#     for i in range(len(adj_pred)):
#         if adj_pred[i] - truth[i] != 0:
#             error_data.append(i)
#     return error_data

# errors = find_error(adj_pred, final_test_set)

# def filter_errors_img(images, error_list):
#     error_img_ds = []
#     for i in range(len(error_list)):
#         error_img_ds.append(images[error_list[i]])
#     return error_img_ds

# img_list = filter_errors_img(img_test_set, errors)

# def display_img(img_list, errors):
#     for i in range(len(img_list)):
#         plt.figure(figsize=(8, 8), dpi=80)
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(np.array(img_list[i],np.int32))
#         plt.title(int(predict[errors[i]]))
#         plt.axis('off')
#         if i == 2:
#             break
            
# display_img(img_test_set, errors)

(1055, 1)